In [1]:
import os
import pandas as pd
pd.options.display.max_columns = 250

import machine_learning as ml
from preprocessing import separate_features_target

In [2]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

train = {
    'stylometric' : ['style_train_balanced.csv','style_train_imbalanced.csv'],
    'word2vec' : ['word2vec_train_balanced.csv','word2vec_train_imbalanced.csv']
}
test = {
    'stylometric' : ['style_test_balanced.csv','style_test_imbalanced.csv'],
    'word2vec' : ['word2vec_test_balanced.csv','word2vec_test_imbalanced.csv']
}

# Balanced Dataset

## Import Data

Since Word2Vec features outperformed the TF-IDF features, only those will be used to test the combination with content features.

### Balanced Dataset

In [3]:
style_train_balanced_complete = pd.read_csv(os.path.join(csv_path, train['stylometric'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
style_test_balanced_complete = pd.read_csv(os.path.join(csv_path, test['stylometric'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

word2vec_train_balanced_complete = pd.read_csv(os.path.join(csv_path, train['word2vec'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
word2vec_test_balanced_complete = pd.read_csv(os.path.join(csv_path, test['word2vec'][0]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

In [4]:
style_train_balanced = separate_features_target(style_train_balanced_complete)
style_test_balanced = separate_features_target(style_test_balanced_complete)

word2vec_train_balanced = separate_features_target(word2vec_train_balanced_complete)
word2vec_test_balanced = separate_features_target(word2vec_test_balanced_complete)

### Imbalanced Dataset

In [5]:
style_train_imbalanced_complete = pd.read_csv(os.path.join(csv_path, train['stylometric'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
style_test_imbalanced_complete = pd.read_csv(os.path.join(csv_path, test['stylometric'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

word2vec_train_imbalanced_complete = pd.read_csv(os.path.join(csv_path, train['word2vec'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})
word2vec_test_imbalanced_complete = pd.read_csv(os.path.join(csv_path, test['word2vec'][1]), index_col=0, dtype={'email_class': 'bool', 'email_id': 'int16'})

In [6]:
style_train_imbalanced = separate_features_target(style_train_imbalanced_complete)
style_test_imbalanced = separate_features_target(style_test_imbalanced_complete)

word2vec_train_imbalanced = separate_features_target(word2vec_train_imbalanced_complete)
word2vec_test_imbalanced = separate_features_target(word2vec_test_imbalanced_complete)

# Merging Feature Sets

The simplest way of combining the information of the two different feature sets is to simply merge them into one set and then perform the predictions based on this concatenated set.

## Balanced Dataset

In [7]:
style_content_train_balanced = pd.concat([word2vec_train_balanced['features'], style_train_balanced['features']], axis=1)
style_content_test_balanced = pd.concat([word2vec_test_balanced['features'], style_test_balanced['features']], axis=1)

### Train

#### Logistic Regression

In [8]:
lr_style_content_balanced = ml.train_logistic_regression(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)
lr_style_content_balanced, lr_style_content_balanced_scaler = lr_style_content_balanced['model'], lr_style_content_balanced['scaler']

Train accuracy: 0.9944320712694877


/home/ichanis/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
dt_style_content_balanced = ml.train_decision_tree(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9862657757980697


In [10]:
rf_style_content_balanced = ml.train_random_forest(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9844097995545658


In [11]:
gb_style_content_balanced = ml.train_gradient_boost(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1)

Train accuracy: 0.9974016332590943


In [12]:
nb_style_content_balanced = ml.train_naive_bayes(style_content_train_balanced, style_train_balanced['target'], show_train_accuracy=1, remove_negatives=True)
nb_style_content_balanced, nb_style_content_balanced_scaler = nb_style_content_balanced['model'], nb_style_content_balanced['scaler']

Train accuracy: 0.9510022271714922


### Results

In [13]:
models = [lr_style_content_balanced, dt_style_content_balanced, rf_style_content_balanced, gb_style_content_balanced, nb_style_content_balanced]
names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting Tree', 'Naive Bayes']

In [14]:
results_style_content_balanced = ml.multi_model_results(models, names, style_content_test_balanced, style_test_balanced['target'], lr_style_content_balanced_scaler, nb_style_content_balanced_scaler)

In [15]:
results_style_content_balanced

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
Logistic Regression,0.965875,0.955182,0.979885,0.967376,0.049080,0.020115,0.965403
Decision Tree,0.951039,0.951289,0.954023,0.952654,0.052147,0.045977,0.950938
Random Forest,0.968843,0.957983,0.982759,0.970213,0.046012,0.017241,0.968373
Gradient Boosting Tree,0.967359,0.960452,0.977011,0.968661,0.042945,0.022989,0.967033
Naive Bayes,0.936202,0.957958,0.916667,0.936858,0.042945,0.083333,0.936861


## Imbalanced Dataset

In [16]:
style_content_train_imbalanced = pd.concat([word2vec_train_imbalanced['features'], style_train_imbalanced['features']], axis=1)
style_content_test_imbalanced = pd.concat([word2vec_test_imbalanced['features'], style_test_imbalanced['features']], axis=1)

### Train

#### Logistic Regression

In [17]:
lr_style_content_imbalanced = ml.train_logistic_regression(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)
lr_style_content_imbalanced, lr_style_content_imbalanced_scaler = lr_style_content_imbalanced['model'], lr_style_content_imbalanced['scaler']

Train accuracy: 0.9927692931477227


/home/ichanis/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
dt_style_content_imbalanced = ml.train_decision_tree(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9833085552101636


In [19]:
rf_style_content_imbalanced = ml.train_random_forest(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9846600892012434


In [20]:
gb_style_content_imbalanced = ml.train_gradient_boost(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1)

Train accuracy: 0.9947965941343425


In [21]:
nb_style_content_imbalanced = ml.train_naive_bayes(style_content_train_imbalanced, style_train_imbalanced['target'], show_train_accuracy=1, remove_negatives=True)
nb_style_content_imbalanced, nb_style_content_imbalanced_scaler = nb_style_content_imbalanced['model'], nb_style_content_imbalanced['scaler']

Train accuracy: 0.9080281119070145


### Results

In [22]:
models = [lr_style_content_imbalanced, dt_style_content_imbalanced, rf_style_content_imbalanced, gb_style_content_imbalanced, nb_style_content_imbalanced]
names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting Tree', 'Naive Bayes']

In [23]:
results_style_content_imbalanced = ml.multi_model_results(models, names, style_content_test_imbalanced, style_test_imbalanced['target'], lr_style_content_imbalanced_scaler, nb_style_content_imbalanced_scaler)

In [24]:
results_style_content_imbalanced

,Accuracy,Precision,Recall,F1 Score,False Positive Rate,False Negative Rate,Area Under ROC Curve
Logistic Regression,0.988108,0.940252,0.922840,0.931464,0.005628,0.077160,0.958606
Decision Tree,0.977838,0.903333,0.836420,0.868590,0.008590,0.163580,0.913915
Random Forest,0.979459,0.955882,0.802469,0.872483,0.003555,0.197531,0.899457
Gradient Boosting Tree,0.988649,0.946203,0.922840,0.934375,0.005036,0.077160,0.958902
Naive Bayes,0.912703,1.000000,0.003086,0.006154,0.000000,0.996914,0.501543


Comparing these with the content-only baseline, it is obvious that there is a at least a small improvement in the balanced dataset and a more significant improvement in the imbalanced dataset.<br>
It seems that the extra features are helpful in order to achieve better accuracy on the bigger dataset.

Another interesting observation is that the false negative rate of the two best performing algorithms is the same. This means that they both did not detect the same number of phishing emails (25).

In [25]:
lr_dt_predictions = ml.results_by_id([lr_style_content_imbalanced, gb_style_content_imbalanced], ['lr', 'gb'], pd.concat([style_test_imbalanced_complete[['email_id', 'email_class']],  style_content_test_imbalanced], axis=1), style_test_imbalanced_complete['email_id'], lr_style_content_imbalanced_scaler)
lr_dt_predictions[(lr_dt_predictions['True Class'] == True) & ((lr_dt_predictions['lr'] == False) & (lr_dt_predictions['gb'] == False))].shape[0]

15

15 of them were missclassified by both algorithms.